In [71]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers.least_squares import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [72]:
from helpers.proj1_helpers import *
# Aya : '/Users/mac/Documents/GitHub/ml-project-1-aaa_project1/data/train.csv' 
DATA_TRAIN_PATH = '/home/adeye/Documents/EPFL/MA1/ML_course/projects/ml-project-1-aaa_project1/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [73]:
from helpers.gradient_descent import *
from helpers.least_squares import *
from helpers.stochastic_gradient_descent import *
from helpers.ridge import *
from helpers.processing import *

## Pre-processing our DataSet

In [74]:
jet_0,label_0,jet_1,label_1,jet_2_3, label_2_3= pre_process_data_pipeline(tX,y)

In [75]:
print(tX.shape)
tX,_,_ = standardize(tX)
print(tX)

(250000, 30)
[[ 0.72343805  0.52317289  0.62968263 ...  0.40687538  0.3983056
   0.66583024]
 [ 0.77526501  0.56264922  0.64215782 ... -1.90048197 -1.90048197
   0.51064925]
 [-1.90048197  0.77811391  0.69456379 ... -1.90048197 -1.90048197
   0.50609331]
 ...
 [ 0.64728354  0.54363654  0.57896063 ... -1.90048197 -1.90048197
   0.50088224]
 [ 0.62304826  0.44867927  0.56275072 ... -1.90048197 -1.90048197
   0.40401494]
 [-1.90048197  0.57184875  0.56740816 ... -1.90048197 -1.90048197
   0.40401494]]


## Learning Algorithms After Cleaning Our DataSet

### Least Squares With Gradient Descent

In [82]:
jet_0,label_0,jet_1,label_1,jet_2_3, label_2_3= groupy_by_jet_num(tX,y)
    
remove_outliers(tX)
init_weights = np.ones((30,))
print(y.shape)
print(tX.shape)

weights,loss =  gradient_descent(y, tX,init_weights , 50000, .01)

print(weights)
print("{:e}".format(loss))

(250000,)
(250000, 30)
[ 0.10912711 -2.86291437 -0.17247185  0.56711906  0.0285999   0.30646842
 -0.03928044  0.11530093 -0.32190172  0.54930635 -0.01166931  0.18092694
  0.02335162  2.09766599  0.0748292   0.06904471 -0.25909315  0.07442551
  0.08421695  0.4680382   0.08175617 -0.18742818  0.05048302  0.01565893
  0.01003774  0.01155916 -0.33112311  0.0094454   0.00545456 -1.13471646]
3.606863e-01


## Least Squares with OLS

In [77]:
weight,loss= least_squares(y,tX)
print(compute_squared_loss(y,tX,weight))

0.33944681722221454


## Least Squares With Stochastic Gradient Descent 

In [78]:
remove_outliers(tX)
init_weights = np.ones((30,))
print(y.shape)
print(tX.shape)

weights,loss =  stochastic_gradient_descent(y, tX,init_weights , 500, 0.01, batch_size=1)

print(weights)
print(loss)

(250000,)
(250000, 30)
[ 0.1448321   0.05714019 -0.0331567   0.05921305  0.12674932 -0.14524506
  0.12449748  0.27470122  0.15879255 -0.45332627  0.27633904  0.28399195
  0.13178061  0.17486661  0.28559417  0.28369308  0.10154469  0.28483899
  0.28425031  0.1353438   0.28539281 -0.70540598  0.27419878 -0.04558516
  0.17789485  0.17573642 -0.02374879  0.13345702  0.13272738 -0.16201421]
0.2898864373438115


## Least Squares With Ridge Regression 

In [31]:
weights, loss = ridge(y, tX, 0.025)
print("{:e}".format(np.sqrt(2*loss)))

2.065204e+05


## Parameter Tuning

## Generate predictions and save ouput in csv format for submission:

In [80]:
DATA_TEST_PATH = '/home/adeye/Documents/EPFL/MA1/ML_course/projects/ml-project-1-aaa_project1/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)



In [81]:
OUTPUT_PATH = '/home/adeye/Documents/EPFL/MA1/ML_course/projects/ml-project-1-aaa_project1/output.csv' 
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)